**실습 목표<br><br>
•추천시스템의 개념과 목적을 이해한다.<br>
•Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.<br>
•음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.<br>
•추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다<br>
•유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.<br>
•새로운 데이터셋으로 직접 추천 모델을 만들어 본다.**

**목차<br><br>
1.들어가며<br>
2.데이터 탐색하기와 전처리<br>
3.사용자의 명시적/암묵적 평가<br>
4.Matrix Factorization(MF)<br>
5.CSR(Compressed Sparse Row) Matrix<br>
6.MF 모델 학습하기<br>
7.비슷한 아티스트 찾기 + 유저에게 추천하기<br>
8.프로젝트 - MovieLens 영화 추천 실습**

In [247]:
import pandas as pd

In [248]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [249]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [250]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [251]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [252]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


**•MF models**

In [253]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [254]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
count_unique = ratings['count'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
count_to_idx = {v:k for k,v in enumerate(count_unique)}

In [255]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_count_data = ratings['count'].map(count_to_idx.get).dropna()
if len(temp_count_data) == len(ratings):
    print('count column indexing OK!!')
    ratings['count'] = temp_count_data
else:
    print('count column indexing Fail!!')

ratings

user_id column indexing OK!!
count column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,1193,0,978300760
1,0,661,1,978302109
2,0,914,1,978301968
3,0,3408,2,978300275
4,0,2355,0,978824291
...,...,...,...,...
1000203,6038,1090,1,956715518
1000205,6038,1094,0,956704887
1000206,6038,562,0,956704746
1000207,6038,1096,2,956715648


In [256]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [257]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_artist = ratings['count'].nunique()

csr_data = csr_matrix((ratings.timestamp, (ratings.user_id, ratings['count'])), shape= (num_user, num_artist))
csr_data

<6039x3 sparse matrix of type '<class 'numpy.longlong'>'
	with 18061 stored elements in Compressed Sparse Row format>

In [258]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3x6039 sparse matrix of type '<class 'numpy.longlong'>'
	with 18061 stored elements in Compressed Sparse Column format>

In [259]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

**•ratings에 있는 유니크한 영화 개수<br>
•rating에 있는 유니크한 사용자 수<br>
•가장 인기 있는 영화 30개(인기순)**

In [260]:
# 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [261]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [262]:
famous_movie_count = ratings.groupby('movie_id')['count'].count()
famous_movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: count, dtype: int64

In [263]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(3952)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [264]:
using_cols = ['movie_id', 'count']
ratings = ratings[using_cols]
ratings.head(10)

,movie_id,count
0,1193,0
1,661,1
2,914,1
3,3408,2
4,2355,0
5,1197,1
6,1287,0
7,2804,0
8,594,2
9,919,2


In [265]:
Mega_data = pd.merge(ratings, movies, on='movie_id') #merge를 활용해서 두 데이터를 합쳤습니다.
Mega_data

,movie_id,count,title,genre
0,1193,0,One Flew Over the Cuckoo's Nest (1975),Drama
1,1193,0,One Flew Over the Cuckoo's Nest (1975),Drama
2,1193,2,One Flew Over the Cuckoo's Nest (1975),Drama
3,1193,2,One Flew Over the Cuckoo's Nest (1975),Drama
4,1193,0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...
836473,3607,0,One Little Indian (1973),Comedy|Drama|Western
836474,3026,2,Slaughterhouse (1987),Horror
836475,690,1,"Promise, The (Versprechen, Das) (1994)",Romance
836476,2909,2,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [266]:
famous_movie = Mega_data.groupby('title')['count'].count() #columns중에서 title과 count만 뽑아서 내림차순으로 정리했습니다.
famous_movie.sort_values(ascending=False).head(30) #가장 인기 있는 영화 30개(인기순)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

**내가 선호하는 영화를 5가지 골라서 추가해 줍시다**

In [267]:
my_pic_movie = ['contact', 'Gattaca', 'THE WAILING', 'Oldboy', 'your name']

my_pic_movie_count = pd.DataFrame({'title': my_pic_movie, 'count':[10]*5})

if not Mega_data.isin({'movie_id':['contact', 'Gattaca', 'THE WAILING', 'Oldboy', 'your name']})['movie_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    Mega_data = Mega_data.append(my_pic_movie_count) # 위에 임의로 만든 my_pic_movie 데이터를 추가해 줍니다. 

Mega_data.tail()

,movie_id,count,title,genre
0,NaN,10,contact,NaN
1,NaN,10,Gattaca,NaN
2,NaN,10,THE WAILING,NaN
3,NaN,10,Oldboy,NaN
4,NaN,10,your name,NaN


**CSR matrix를 직접 만들어 봅시다**

In [268]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [269]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [270]:
movie_unique = Mega_data['movie_id'].unique()
title_unique = Mega_data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [271]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(title_to_idx['Oldboy'])

3631


In [272]:
Mega_data

,movie_id,count,title,genre
0,1193.0,0,One Flew Over the Cuckoo's Nest (1975),Drama
1,1193.0,0,One Flew Over the Cuckoo's Nest (1975),Drama
2,1193.0,2,One Flew Over the Cuckoo's Nest (1975),Drama
3,1193.0,2,One Flew Over the Cuckoo's Nest (1975),Drama
4,1193.0,0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...
0,NaN,10,contact,NaN
1,NaN,10,Gattaca,NaN
2,NaN,10,THE WAILING,NaN
3,NaN,10,Oldboy,NaN


In [277]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = Mega_data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_user_data) == len(Mega_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('movie_id column indexing Fail!!')
    Mega_data['movie_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('movie_id column indexing OK!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_user_id_data = Mega_data['count'].map(count_to_idx.get).dropna()
if len(temp_count_data) == len(Mega_data):
    print('count column indexing Fail!!')
    Mega_data['count'] = temp_user_id_data
else:
    print('count column indexing Ok!!')

Mega_data

movie_id column indexing OK!!
count column indexing Ok!!


,movie_id,count,title,genre
0,1193.0,0,One Flew Over the Cuckoo's Nest (1975),Drama
1,1193.0,0,One Flew Over the Cuckoo's Nest (1975),Drama
2,1193.0,2,One Flew Over the Cuckoo's Nest (1975),Drama
3,1193.0,2,One Flew Over the Cuckoo's Nest (1975),Drama
4,1193.0,0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...
0,NaN,10,contact,NaN
1,NaN,10,Gattaca,NaN
2,NaN,10,THE WAILING,NaN
3,NaN,10,Oldboy,NaN


In [274]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=120, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [275]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3x6039 sparse matrix of type '<class 'numpy.longlong'>'
	with 18061 stored elements in Compressed Sparse Column format>

In [276]:
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]